In [1]:
import numpy as np
import matplotlib.pyplot as plt

We use the barrier method to solve the following QP.

\begin{align}
\min_v & &v^T Q v + p^T v \\
s.t. &\ &Av \preceq b
\end{align}

in variable $v \in \mathbb{R}^n$, where $Q \succeq 0$

In [12]:
def centering_step(Q, p, A, b, t, v0, eps, max_iter = 100):
    """This function implements the centering step for the barrier method.
    
    We compute x*(t) by minimizing tf0 + phi. The x*(t) is calculated by using Newton method
    
    Params:
        Q    (np.array): a matrix of size nxn, which defines the problem
        p    (np.array): a matrix of size nx1, which defines the problem
        A    (np.array): a matrix of size dxn, which defines the problem
        b    (float)   : defines the problem
        
        t    (float)   : optimization parameter, which will be updated after each iteration
        v0   (np.array): a matrix of size nx1, which is the initialization for v
        esp  (float)   : target precision
    """
    
    v = v0
    v0 = np.zeros_like(v)
    i = 0
    
    while i < max_iter and np.linalg.norm(v - v0) > eps:
        v0 = v
        # b - Av
        residu = (b - np.dot(A, v0)).reshape((-1, 1))
        # L(v)
        fv = ((2 * np.dot(Q, v) + p) + (A / residu)).sum(1).reshape((-1, 1))
        # Jac(L(v))
        J = 2 * t * Q - (np.dot(A.T, A)[:, :, np.newaxis] / residu.reshape((1, 1, -1))).sum(2)
        # v_new = v - Jn v'
        v = v0 - np.dot(np.dot(np.linalg.inv(np.dot(J.T, J)), J.T), fv)
        i = i + 1
        
    return v      

In [13]:
def barr_method(Q, p, A, b, v0, eps, max_iter = 1000):
    """This function implements the barrier method for a QP problem.
    
    Params:
        Q    (np.array): a matrix of size nxn, which defines the problem
        p    (np.array): a matrix of size nx1, which defines the problem
        A    (np.array): a matrix of size dxn, which defines the problem
        b    (float)   : defines the problem
        
        v0   (np.array): a matrix of size nx1, which is the initialization for v
        esp  (float)   : target precision
    """
    
    t, i, mu, m = 1, 0, 2, A.shape[0]
    
    while i < max_iter:
        
        v0 = centering_step(Q, p, A, b, t, v0, eps)
        if (m / t) < eps:
            break
        else:
            t = mu * t
            
    return v0

In [16]:
from sklearn.datasets import load_boston

In [22]:
boston = load_boston()
X, Y, class_names = boston["data"], boston["target"], boston["feature_names"]